### First Let's import required libraries

In [93]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
from pandas.io.json import json_normalize  # transform json files to pandas dataframes
from geopy.geocoders import Nominatim # 
import numpy as np
import csv
!pip install folium
import folium

print('All modules imported')

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
All modules imported


## Let's start scraping the wikipedia page

In [94]:
# The wikipedia site link
site_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Get the source code html data from the website

In [95]:
source = requests.get(site_link).text

Lets Use BeautifulSoup to parse it

In [96]:
soup = BeautifulSoup(source, 'lxml')

#print(soup.prettify())

Next let's get the table that contains the data we want to scrape

In [97]:
My_table = soup.find('table',{'class':'wikitable sortable'})

Let's view the table data

In [98]:
# You can uncomment below to view table

#My_table

we can see that all the data we want are between the $<td>$ brackets, let's get the data between the td brackets

In [99]:
links = My_table.find_all('td')

In [100]:
# You can uncomment below to view links

#print(links)

Next let's loop through links and extract only the text elements

In [101]:
text_links = []

for link in links:
    text_links.append(link.text)

# Uncomment below to view text_links

# text_links

### let's clean and process the table elements

Let's clean the links and keep only rows with Borough. Out of which we shall rename rows without Neighborhood as Boroughs

In [102]:
cleaned_links = []

while True:
    
    if len(text_links) < 3:
        break
    
    sub = text_links[:3]
    # If 'Not ' in borough then skip that row of data
    if 'Not ' in sub[1]:
        text_links = text_links[3:]
    else:
        cleaned_links.append(text_links[:3])
        
        # Let's strip off the \n at the end of each neighborhood data
        cleaned_links[-1][-1] = cleaned_links[-1][-1].strip('\n')
        
        # If the Borough is available but the Neighborhood is missing
        # make Neighborhood same as Borough
        if 'Not ' in cleaned_links[-1][-1]:
            cleaned_links[-1][-1] = cleaned_links[-1][-2]
        text_links = text_links[3:]

# Uncomment below to view cleaned_links        

#cleaned_links

lets check the length of the cleaned links

In [103]:
len(cleaned_links)

211

Next let's add the neighborhood data of each duplicate Postal Codes together to the first instance or row that contains the PostalCode

In [104]:
link = []
for i in range(len(cleaned_links)):
    x = cleaned_links[i][0]
    if x in link:
        cleaned_links[link.index(x)][-1] += ', ' + cleaned_links[i][-1]
    link.append(x)
cleaned_links

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront, Regent Park'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights, Lawrence Manor'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", "Queen's Park"],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge, Malvern'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens, Parkview Hill'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson, Garden District'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M9B',
  'Etobicoke',
  'Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park'],
 ['M9B', 'Etobicoke', 'Islington'],
 ['M9B', 'Etobicoke', 'Martin Grove'],
 ['M9B', 'Etobicoke', 'Princess Gardens'],
 ['M9B', 'Etobicoke', 'West Deane Park'],
 ['

Next let's pass the cleaned _links to a data frame and set index to postal code so that we can easily work on it

In [105]:
df = pd.DataFrame(cleaned_links, columns=['PostalCode','Borough','Neighborhood'])
df.index= df.PostalCode

In [106]:
# Let's view the data frame
df.head()

,PostalCode,Borough,Neighborhood
PostalCode,,,
M3A,M3A,North York,Parkwoods
M4A,M4A,North York,Victoria Village
M5A,M5A,Downtown Toronto,"Harbourfront, Regent Park"
M5A,M5A,Downtown Toronto,Regent Park
M6A,M6A,North York,"Lawrence Heights, Lawrence Manor"


Next let's Use the pandas duplicate method to drop duplicate index

In [107]:
df = df.loc[~df.index.duplicated(keep='first')]

In [109]:
df.head()

,PostalCode,Borough,Neighborhood
PostalCode,,,
M3A,M3A,North York,Parkwoods
M4A,M4A,North York,Victoria Village
M5A,M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,M7A,Queen's Park,Queen's Park


Next let's reset index so it is like the example on the assignment page

In [111]:
df = df.reset_index(drop=True)

Finally let's view the first 20 rows and print the shape

In [112]:
df.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [113]:
df.shape

(103, 3)